# Θ-extraction from optical conductivity (production demo)
Pipeline: $\sigma(\omega,T) \to M(\omega) \to \pi(\omega) \to E,S \to \Theta(T) \to \Theta_c/T_c$.

**What's included**
- Data loader (CSV) + robust synthetic generator
- Memory-function extraction + KK/causality diagnostics
- Spectral measure, entropy/energy, Θ extraction
- Multi-method Θ_c detection (consensus)
- Classification & validation gates (PASS/FAIL)
- Plots: σ(ω,T), M(ω), π(ω), Θ(T), diagnostics


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from extract_theta_from_optical_v2 import (
    extract_theta_full_pipeline, MATERIALS, generate_dummy_data, compute_memory_function,
    spectral_measure, extract_theta
)
ROOT = Path('.')


## 1) Generate synthetic σ(ω,T) and preview

In [ ]:
material = 'LSCO_x015'
Ts = np.array([100,150,200,250,300], dtype=float)
# Use synthetic data for a clean, reproducible demo
syn_data = {}
for T in Ts:
    w, s1, s2 = generate_dummy_data(material, T)
    syn_data[T] = (w, s1, s2)

# Plot σ1 and σ2 at two representative T
fig, axes = plt.subplots(1,2, figsize=(10,4))
for T,ax in zip([Ts[0], Ts[-1]], axes):
    w,s1,s2 = syn_data[T]
    ax.plot(w, s1, label=r'$\sigma_1$')
    ax.plot(w, s2, label=r'$\sigma_2$')
    ax.set_title(f"σ(ω) at T={T:.0f} K")
    ax.set_xlabel('ω [eV]'); ax.set_ylabel('σ [arb.]')
    ax.legend()
plt.tight_layout(); plt.show()


## 2) Memory function, spectral measure and Θ at a single T

In [ ]:
T0 = Ts[2]
w, s1, s2 = syn_data[T0]
mat = MATERIALS[material]
M1, M2, Mdiag = compute_memory_function(w, s1, s2, mat.omega_p)
pi = spectral_measure(w, s1, method='f-sum')
Theta, Tdiag = extract_theta(w, pi, M1, energy_method='canonical')
print('Diagnostics (M):', Mdiag)
print('Θ at T=%.0f K = %.6f eV' % (T0, Theta))

fig, ax = plt.subplots(1,3, figsize=(14,3.8))
ax[0].plot(w, M1); ax[0].plot(w, M2)
ax[0].set_title('Memory function M(ω)'); ax[0].set_xlabel('ω [eV]'); ax[0].set_ylabel('Re/Im M')
ax[1].plot(w, pi)
ax[1].set_title('Spectral measure π(ω)'); ax[1].set_xlabel('ω [eV]')
ax[2].plot(w, s1); ax[2].plot(w, s2)
ax[2].set_title('σ(ω) for reference'); ax[2].set_xlabel('ω [eV]')
plt.tight_layout(); plt.show()


## 3) Full pipeline, Θ(T) curve, Θ_c detection, classification & validation

In [ ]:
res = extract_theta_full_pipeline(material, Ts, use_dummy=True, energy_method='canonical', verbose=False)
print('SUMMARY')
print(json.dumps({k:res[k] for k in ['material','Tc','Theta_c','ratio','classification','confidence','validation']}, indent=2))

# Plot Θ(T)
fig, ax = plt.subplots(figsize=(5,4))
ax.plot(res['T_array'], res['Theta_array'], marker='o')
ax.axvline(res['Tc'], ls='--')
ax.set_xlabel('T [K]'); ax.set_ylabel('Θ [eV]'); ax.set_title('Θ(T)')
plt.tight_layout(); plt.show()

# Diagnostics dashboard (text)
from pprint import pprint
print('\nΘ_c diagnostics:'); pprint(res['Theta_c_diagnostics'])
print('\nClassification diagnostics:'); pprint(res['classification_diagnostics'])
print('\nValidation gate:'); pprint(res['validation'])


## 4) Falsification tests
- **ε(ω)** sensitivity: canonical vs memory vs corrected energy models
- **Θ_c methods** agreement (A/B/C) → std/mean
- **PASS/FAIL** gates

In [ ]:
# ε(ω) sensitivity
methods = ['canonical','memory','corrected']
vals = {}
for m in methods:
    res_m = extract_theta_full_pipeline(material, Ts, use_dummy=True, energy_method=m, verbose=False)
    vals[m] = {'Theta_c': res_m['Theta_c'], 'ratio': res_m['ratio'], 'agreement': res_m['Theta_c_diagnostics']['agreement']}
print('Energy model sensitivity:')
print(json.dumps(vals, indent=2))

# Aggregate PASS/FAIL
status = res['validation']['status']
print('\nOVERALL STATUS:', status)
